In [86]:
base_path = "/content/drive/My Drive/yl_tez"

In [87]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
!pip install transformers==4.41.2
!pip install datasets
!pip install trl

In [89]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

In [90]:
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)
veri_seti_adi =  "random_templated_sohbetler20000.csv"
data_filepath = os.path.join(base_path, veri_seti_adi)

In [91]:
# Veri dosyasını oku
df = pd.read_csv(data_filepath)

In [92]:
print(df.head().to_markdown())

|    | text                                                                                                                                                                                                                                                                                                                                         |
|---:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Durum: Ben hipertansiyon hastası değilim.  Bende kalp hastalığı yok.  Benim kilom biraz fazla.  Ben yaşlı bir insanım.  Ben bir kadınım.  Ben hayatım boyunca asla sigara içmedim.  Ben HbA1c seviyeme göre prediyabet durumundayım.  Benim kan şekerim aşırı derecede yüksek.  Bilgi: Sende diyabet hastalığı mevcut

In [93]:
len(df)

1000

In [94]:
eval_dataset = Dataset.from_pandas(df)

In [95]:
eval_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [103]:
# Eğitim argümanlarını tanımla (sadece değerlendirme için gerekli olanlar)
training_args = SFTConfig(
    output_dir=model_yolu,  # Modelin kaydedileceği dizin
    report_to="none",  # Raporlama aracı
    per_device_eval_batch_size=1,  # Değerlendirme batch boyutu (azaldıkça vram kullanımı azalır)
    dataloader_drop_last=True,  # Son batch'ı atla
    eval_accumulation_steps=100,  # Değerlendirme adım sayısı (azaldıkça vram kullanımı azalır)
    max_seq_length=512,  # Maksimum girdi uzunluğu ÖNEMLİ
)

In [98]:
!pip install evaluate
!pip install rouge_score
!pip install bert_score

In [104]:
import evaluate
import numpy as np

In [108]:
# Metrikleri yükleme
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
perplexity_metric = evaluate.load("perplexity")
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")

# compute_metrics fonksiyonunu tanımlama
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # -100 değerlerini 0 ile değiştir
    labels = np.where(labels == -100, 0, labels)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # BLEU skoru hesaplama
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    
    # ROUGE skoru hesaplama
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    
    # Perplexity (şaşırtıcılık) skoru hesaplama
    perplexity_result = perplexity_metric.compute(predictions=logits, labels=labels)
    
    # METEOR skoru hesaplama
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    
    # BERTScore skoru hesaplama
    bertscore_result = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang='en')
    
    # Tüm metrikleri birleştirme
    metrics = {
        "bleu_sonuclari": bleu_result,
        "rouge": rouge_result,
        "perplexity": perplexity_result,
        "meteor": meteor_result,
        "bertscore": bertscore_result
    }
    
    return metrics

In [109]:
# Eğitici (Trainer) tanımla (yalnızca değerlendirme için gerekli olanlar)
trainer = SFTTrainer(
    model_yolu,
    args=training_args,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    compute_metrics=compute_metrics,  # compute_metrics fonksiyonunu geçirin
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Değerlendirmeyi başlat
results = trainer.evaluate()

In [ ]:
print(results)

In [ ]:
import json
# Değerlendirme sonuçlarını kaydetme
results_file = os.path.join(model_yolu, "evaluation_results.json")
with open(results_file, "w") as f:
    json.dump(results, f)